In [117]:
def best_perf_syms_sets_lookback_slices(sets_lookback_slices, verbose=False):
    # grp_top_set_syms_n_freq is a list of lists of top_set_syms_n_freq, e.g.
    #   [[('AGY', 7), ('PCG', 7), ('KDN', 6), ..., ('CYT', 3)],
    #    [('FCN', 9), ('HIG', 9), ('SJR', 8), ..., ('BFH', 2)]]
    #   where each list is the best performing symbols from a lb_slices, e.g.
    #     [(483, 513, 523), (453, 513, 523), (393, 513, 523)]
    grp_top_set_syms_n_freq = (
        []
    )  # list of lists of top_set_symbols_n_freq, there are n_samples lists in list
    grp_top_set_syms = []  # grp_top_set_syms_n_freq without the frequency count

    # lb_slices, e.g  [(483, 513, 523), (453, 513, 523), (393, 513, 523)],
    #  is one max_lookback_slice, e.g. (393, 513, 523), along with
    #  the remaining slices of the days_lookbacks, e.g. (483, 513, 523), (453, 513, 523)
    for i, lb_slices in enumerate(sets_lookback_slices):
        print(
            f"\n########## {i + 1} of {len(sets_lookback_slices)} lb_slices in sets_lookcak_slices ##########"
        )
        # unsorted list of the most frequent symbols in performance metrics of the lb_slices
        grp_most_freq_syms = []
        for j, lb_slice in enumerate(lb_slices):  # lb_slice, e.g. (246, 276, 286)
            iloc_start_train = lb_slice[0]  # iloc of start of training period
            iloc_end_train = lb_slice[1]  # iloc of end of training period
            iloc_start_eval = iloc_end_train  # iloc of start of evaluation period
            iloc_end_eval = lb_slice[2]  # iloc of end of evaluation period
            n_days = (
                iloc_end_train - iloc_start_train
            )  # one of the lookback-days in lookback-slice
            d_eval = iloc_end_eval - iloc_start_eval

            _df = df.iloc[iloc_start_train:iloc_end_train]
            date_start_df_train = _df.index[0].strftime("%Y-%m-%d")
            date_end_df_train = _df.index[-1].strftime("%Y-%m-%d")

            if verbose:
                print(
                    f"days lookback:       {n_days},  {j + 1} of {len(days_lookbacks)} days_lookbacks: {days_lookbacks}"
                )
                print(f"lb_slices:           {lb_slices}")
                print(f"lb_slice:            {lb_slice}")
                print(f"days eval:           {d_eval}")
                print(f"iloc_start_train:    {iloc_start_train}")
                print(f"iloc_end_train:      {iloc_end_train}")
                print(f"date_start_df_train: {date_start_df_train}")
                print(f"date_end_df_train:   {date_end_df_train}")

            perf_ranks, most_freq_syms = rank_perf(_df, n_top_syms=n_top_syms)
            # unsorted list of the most frequent symbols in performance metrics of the lb_slices
            grp_most_freq_syms.append(most_freq_syms)
            if verbose:
                # n_days of r_CAGR/UI, r_CAGR/retnStd, r_retnStd/UI
                print(f"perf_ranks: {perf_ranks}")
                # most common symbols of perf_ranks
                print(f"most_freq_syms: {most_freq_syms}")
                print(f"+++ finish lookback slice {n_days} +++\n")

        if verbose:
            print(f"grp_most_freq_syms: {grp_most_freq_syms}")
            # grp_most_freq_syms a is list of lists of tuples of
            #  the most-common-symbols symbol:frequency cumulated from
            #  each days_lookback
            print(f"**** finish lookback slices {lb_slices} ****\n")

        # flatten list of lists of (symbol:frequency)
        flat_grp_most_freq_syms = [
            val for sublist in grp_most_freq_syms for val in sublist
        ]
        # return "symbol, frequency" pairs of the most frequent symbols, i.e. best performing symbols,
        #  in flat_grp_most_freq_syms. The paris are sorted in descending frequency.
        set_most_freq_syms = grp_tuples_sort_sum(flat_grp_most_freq_syms, reverse=True)
        # get the top n_top_syms of the most frequent "symbol, frequency" pairs
        top_set_syms_n_freq = set_most_freq_syms[0:n_top_syms]
        # get symbols from top_set_syms_n_freq, i[0] = symbol, i[1]=symbol's frequency count
        top_set_syms = [i[0] for i in top_set_syms_n_freq[syms_start:syms_end]]

        # grp_top_set_syms_n_freq is a list of lists of top_set_syms_n_freq, e.g.
        #   [[('AGY', 7), ('PCG', 7), ('KDN', 6), ..., ('CYT', 3)],
        #    [('FCN', 9), ('HIG', 9), ('SJR', 8), ..., ('BFH', 2)]]
        #   where each list is the best performing symbols from a lb_slices, e.g.
        #     [(483, 513, 523), (453, 513, 523), (393, 513, 523)]
        grp_top_set_syms_n_freq.append(top_set_syms_n_freq)
        grp_top_set_syms.append(top_set_syms)

        if verbose:
            print(
                f"top {n_top_syms} ranked symbols and frequency from set {lb_slices}:\n{top_set_syms_n_freq}"
            )
            print(
                f"top {n_top_syms} ranked symbols from set {lb_slices}:\n{top_set_syms}"
            )
            print(
                f"===== finish top {n_top_syms} ranked symbols from days_lookback set {lb_slices} =====\n\n"
            )

    return grp_top_set_syms_n_freq, grp_top_set_syms, date_end_df_train



In [118]:
import pandas as pd
from yf_utils import random_slices, lookback_slices
from yf_utils import rank_perf, grp_tuples_sort_sum, top_set_sym_freq_cnt
from myUtils import pickle_load, pickle_dump

pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 11)
pd.set_option("display.max_colwidth", 26)
# pd.set_option("display.width", 145)
pd.set_option("display.width", 280)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

fp_df_close_clean = "df_close_clean"
fp_df_picks = "df_picks"
fp_l_days_to_drop = "l_days_to_drop"
# pickle file of dates missing from df_picks
fp_dates_missing_in_df_picks = 'dates_missing_in_df_picks'

In [119]:
df_picks = pickle_load(path_data_dump, fp_df_picks)
print(f'df_picks:\n{df_picks}')

df_picks:
    date_end_df_train  max_days_lookbacks     days_lookbacks sym_freq_15 sym_freq_14  ...                 sym_freq_6                 sym_freq_5                 sym_freq_4                 sym_freq_3 sym_freq_2
0          2023-11-13                 120      [30, 60, 120]          []          []  ...  ['ANF', 'CAH', 'LRN', ...  ['BTC-USD', 'EDU', 'US...      ['ALL', 'GPS', 'TGH']  ['CHE', 'CLS', 'DELL',...         []
1          2023-11-13                 120  [15, 30, 60, 120]          []          []  ...      ['ANF', 'CAH', 'NRG']  ['BTC-USD', 'EDU', 'US...  ['ALL', 'AMBC', 'CHE',...                         []         []
2          2023-11-10                 120      [30, 60, 120]          []          []  ...  ['ANF', 'CAH', 'LRN', ...        ['BTC-USD', 'ICPT']             ['EDU', 'GPS']  ['AGYS', 'ALL', 'CLS',...         []
3          2023-11-10                 120  [15, 30, 60, 120]          []          []  ...     ['ANF', 'CAH', 'USAC']  ['AGYS', 'BTC-USD', 'I...  ['AMBC', 

In [120]:
df_close_clean = pickle_load(path_data_dump, fp_df_close_clean)
df_current = df_close_clean.copy()
print(f'df_current:\n{df_current}')

df_current:
                     A         AA        AAL        AAP        AAPL  ...       ZION        ZTO         ZTS       ZUMZ        ZWS
Date                                                                 ...                                                        
2017-12-05   62.900620  40.307934  48.216740  93.069862   40.103485  ...  42.263466  14.571017   68.079163  20.650000  24.089575
2017-12-06   62.957947  40.435177  48.353188  89.711449   39.954559  ...  41.466038  14.720368   68.194473  21.049999  23.712727
2017-12-07   63.741459  39.740219  49.591011  89.812660   40.027855  ...  41.922905  14.953729   68.838264  21.150000  23.761038
2017-12-08   64.381660  40.523281  49.727467  92.296982   40.039665  ...  42.305004  14.262982   69.068893  21.200001  23.867332
2017-12-11   64.018562  41.345482  50.000370  92.793831   40.819813  ...  41.482655  14.487009   68.963211  20.950001  23.741716
...                ...        ...        ...        ...         ...  ...        ...  

In [121]:
dates_missing_in_df_picks = pickle_load(path_data_dump, fp_dates_missing_in_df_picks)
print(f'dates_missing_in_df_picks, len({len(dates_missing_in_df_picks)}):\n{dates_missing_in_df_picks}')

dates_missing_in_df_picks, len(26):
['2023-07-17', '2023-07-19', '2023-07-24', '2023-07-28', '2023-07-31', '2023-08-16', '2023-08-21', '2023-08-22', '2023-08-23', '2023-08-24', '2023-08-28', '2023-08-29', '2023-08-30', '2023-08-31', '2023-09-06', '2023-09-07', '2023-09-19', '2023-09-26', '2023-09-27', '2023-10-03', '2023-10-09', '2023-10-10', '2023-10-11', '2023-10-13', '2023-10-16', '2023-10-18']


In [122]:
l_days_to_drop = pickle_load(path_data_dump, fp_l_days_to_drop)
print(f'l_days_to_drop, len({len(l_days_to_drop)}):\n{l_days_to_drop}') 

l_days_to_drop, len(26):
[85, 83, 80, 76, 75, 63, 60, 59, 58, 57, 55, 54, 53, 52, 49, 48, 40, 35, 34, 30, 26, 25, 24, 22, 21, 19]


In [123]:
import pandas as pd
from yf_utils import random_slices, lookback_slices
from yf_utils import rank_perf, grp_tuples_sort_sum, top_set_sym_freq_cnt
from myUtils import pickle_load, pickle_dump

pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 11)
pd.set_option("display.max_colwidth", 16)
pd.set_option("display.width", 145)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

fp_df_close_clean = "df_close_clean"

### Set parameters ######################################################################
# fp_df_eval_results = f'df_eval_results_{run_type}'
fp_df_picks = f"df_picks"

# verbose = False  # True prints more output
verbose = True  # True prints more output

store_results = True
# store_results = False

# n_samples is the number of random samples of df chunks s.t.
#   len(df) = max_days_lookbacks. For getting the current picks
#   n_samples is always 1 since more samples will always yield
#   the same df chunk
n_samples = 1  # only need 1 sample to get the current picks

# for training, the number of days to lookback from iloc max-lookback iloc_end_train
l_days_lookbacks = [[30, 60, 120], [15, 30, 60, 120]]

# e.g sort from [[60, 30, 120], [15, 60, 30, 120]] to [[30, 60, 120], [15, 30, 60, 120]] 
l_sorted_days_lookbacks = []
for days_lookbacks in l_days_lookbacks:
    l_sorted_days_lookbacks.append(sorted(days_lookbacks))

# number of days from iloc_end_train are used to evaluate effectiveness of the training
days_eval = 0

# number of the most-common symbols from days_lookbacks' performance rankings to keep
n_top_syms = 20

# slice starts and ends for selecting the best performing symbols
syms_start = 0
syms_end = 10

# get picks of previous days by dropping the last n rows from df_current
#  drop_last_n_rows = 1 drops the last row from df_current
drop_last_n_rows = 0
#########################################################################

print(f"verbose : {verbose }")
print(f"store_results: {store_results}")
print(f"n_samples: {n_samples}")
print(f"days_lookbacks: {days_lookbacks}")
print(f"days_eval: {days_eval}")
print(f"n_top_syms: {n_top_syms}")
print(f"syms_start: {syms_start}")
print(f"syms_end: {syms_end}")
print(f"fp_df_picks: {fp_df_picks}\n\n")

df_picks = pickle_load(path_data_dump, fp_df_picks)
df_close_clean = pickle_load(path_data_dump, fp_df_close_clean)
df_current = df_close_clean.copy()

verbose : True
store_results: True
n_samples: 1
days_lookbacks: [15, 30, 60, 120]
days_eval: 0
n_top_syms: 20
syms_start: 0
syms_end: 10
fp_df_picks: df_picks




In [124]:
for i, days_lookbacks in enumerate(l_sorted_days_lookbacks):
    # print(f'i: {i}, days_lookbacks: {days_lookbacks}')
    print(f'i: {i}, days_lookbacks: {days_lookbacks} in l_sorted_days_lookbacks: {l_sorted_days_lookbacks}')    

    # TODO changed 
    # print(f'\n\n{"%"*40:<42}{i+1} of {len(l_sorted_days_lookbacks)} days_lookbacks{"%"*40:>42}\n')
    print(f'\n\n{"%"*30:<32}{i+1} of {len(l_sorted_days_lookbacks)} days_lookbacks: {days_lookbacks}{"%"*30:>32}\n')

    # ----- drops df rows, limits df length to max_days_lookbacks 
    # e.g. days_lookbacks: [15, 30, 60, 120] => max_days_lookbacks: 120
    max_days_lookbacks = max(days_lookbacks)
    print(f"max_days_lookbacks: {max_days_lookbacks}\n")

    slice_start = -(max_days_lookbacks + drop_last_n_rows)
    slice_end = -drop_last_n_rows
    if drop_last_n_rows == 0:  # return df with all rows
        df = df_current[slice_start:].copy()
    else:  # return df with dropped drop_last_n_rows rows
        df = df_current[slice_start:slice_end].copy()
    print(f"dropped last {drop_last_n_rows} row(s) from df")

    # TODO added code df.head()
    print(f"df.head(2):\n{df.head(2)}")
    print(f"df.tail(2):\n{df.tail(2)}\n")
    len_df = len(df)
    # len_df_current = len(df_current)
    print(f"len(df): {len(df)}")
    # -----


    # return n_samples slices
    max_lookback_slices = random_slices(
        len_df,
        n_samples=n_samples,
        days_lookback=max(days_lookbacks),
        days_eval=days_eval,
        verbose=False,
    )    
    print(f'max_lookback_slices:\n{max_lookback_slices}')

    # return n_samples * len(days_lookbacks) slices
    sets_lookback_slices = lookback_slices(
        max_slices=max_lookback_slices, days_lookbacks=days_lookbacks, verbose=False
    )
    print(f'sets_lookback_slices:\n{sets_lookback_slices}')

    if verbose:
        print(f"number of max_lookback_slices is equal to n_samples = {n_samples}")
        print(f"max_lookback_slices:\n{max_lookback_slices}\n")
        print(f"days_lookbacks: {days_lookbacks}")
        print(f"sets_lookback_slices:\n{sets_lookback_slices}\n")
        print(f"number of sets in sets_lookback_slices is equal to n_samples = {n_samples}")
        print(
            f'number of tuples in each "set of lookback slices" is equal to len(days_lookbacks): {len(days_lookbacks)}'
        )

    (
        grp_top_set_syms_n_freq,
        grp_top_set_syms,
        date_end_df_train,
    ) = best_perf_syms_sets_lookback_slices(sets_lookback_slices, verbose=verbose)

    print(f'grp_top_set_syms_n_freq:\n{grp_top_set_syms_n_freq}\n')
    print(f'grp_top_set_syms:\n{grp_top_set_syms}\n')
    print(f'date_end_df_train:\n{date_end_df_train}\n')    


    print(f'End of For_Loop  =====================\n\n\n')

i: 0, days_lookbacks: [30, 60, 120] in l_sorted_days_lookbacks: [[30, 60, 120], [15, 30, 60, 120]]


%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%  1 of 2 days_lookbacks: [30, 60, 120]  %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

max_days_lookbacks: 120

dropped last 0 row(s) from df
df.head(2):
                     A         AA    AAL        AAP        AAPL  ...       ZION        ZTO         ZTS   ZUMZ        ZWS
Date                                                             ...                                                    
2023-06-01  115.804932  32.465004  14.70  67.459000  179.610504  ...  27.023617  25.790001  165.240097  15.70  23.091518
2023-06-02  117.757256  34.688904  14.87  66.992935  180.468216  ...  28.390911  26.350000  171.093201  15.47  24.166006

[2 rows x 1274 columns]
df.tail(2):
                     A         AA    AAL        AAP        AAPL  ...       ZION        ZTO         ZTS       ZUMZ        ZWS
Date                                                             ...                

perf_ranks: {'period-30': {'r_CAGR/UI': array(['GPS', 'SHV', 'GBTC', 'CRSP', 'SPR', 'OUT', 'BTC-USD', 'STKL',
       'PI', 'PLNT', 'FTSM', 'TGH', 'LRN', 'AES', 'CMG', 'HASI', 'WING',
       'TNK', 'NFLX', 'NRG'], dtype=object), 'r_CAGR/retnStd': array(['GPS', 'CRSP', 'STKL', 'GBTC', 'OUT', 'SPR', 'PI', 'BTC-USD',
       'AES', 'HASI', 'WING', 'TNK', 'VRNS', 'M', 'SHV', 'PLNT', 'CMG',
       'AMT', 'EXPE', 'NRG'], dtype=object), 'r_retnStd/UI': array(['SHV', 'TGH', 'FTSM', 'FCN', 'LRN', 'GPS', 'PLNT', 'BTC-USD', 'VZ',
       'SPR', 'CBOE', 'DECK', 'CHE', 'SBUX', 'OUT', 'NFLX', 'CMG', 'MCD',
       'TGT', 'GBTC'], dtype=object)}}
most_freq_syms: [('GPS', 3), ('SHV', 3), ('GBTC', 3), ('SPR', 3), ('OUT', 3), ('BTC-USD', 3), ('PLNT', 3), ('CMG', 3), ('CRSP', 2), ('STKL', 2), ('PI', 2), ('FTSM', 2), ('TGH', 2), ('LRN', 2), ('AES', 2), ('HASI', 2), ('WING', 2), ('TNK', 2), ('NFLX', 2), ('NRG', 2), ('VRNS', 1), ('M', 1), ('AMT', 1), ('EXPE', 1), ('FCN', 1), ('VZ', 1), ('CBOE', 1), ('DECK', 1),